In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
import math
from google.colab import drive #Mount Google Drive
import timeit

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
#from keras.callbacks import EarlyStopping
from keras.layers import ConvLSTM2D

In [ ]:
drive.mount('/content/drive')

In [ ]:
#Membaca & modifikasi data traffic density USA & Australia
dataset= pd.read_csv('/content/drive/MyDrive/covid_impact_on_airport_traffic.csv')

#dataset_US = dataset[dataset['Country']=='United States of America (the)']
#df_US = pd.DataFrame(dataset_US.groupby('Date',as_index=True)['PercentOfBaseline'].mean())
#print(df_US)

dataset_AUS = dataset[dataset['Country']=='Australia']
df_AUS = pd.DataFrame(dataset_AUS.groupby('Date',as_index=True)['PercentOfBaseline'].mean())
print(df_AUS)

In [ ]:
#Mengubah Dataframe ke type float (df_US:Amerika & df_AUS: Australia)
#dataset = df_US.values
#dataset = dataset.astype('float32') #Mengubah nilai menjadi float
#df_US=df_US.astype('float32')

#dataset = df_AUS.values
#dataset = dataset.astype('float32')
df_AUS=df_AUS.astype('float')

In [ ]:
#LSTM menggunakan sigmoid and tanh yang sensitif terhadap nilai input sehingga harus di-normalisasi
scaler = MinMaxScaler(feature_range=(0, 1))
#dataset = scaler.fit_transform(dataset)
df_AUS=scaler.fit_transform(df_AUS)

In [ ]:
#Pemecahan data (untuk training & testing) tidak dapat dilakukan secara random dalam peristiwa time series

#Kita bagi 2/3 atau 66% data pertama untuk training dan sisasnya 1/3 untuk testing
#train_size = int(len(dataset) * 0.66)
#test_size = len(dataset) - train_size
#train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
train_size = int(len(df_AUS) * 0.66)
test_size = len(df_AUS) - train_size
train, test = df_AUS[0:train_size,:], df_US[train_size:len(df_US),:]




In [ ]:
#Menyusun Dataset Training & Test dimana X adalah jumlah data 'PercentOfBaseline' pada saat(t, t-1, t-2...) 
#dan Y adalah jumlah data 'PercentOfBaseline' pada saat (t + 1).
#(df_US:Amerika & df_AUS: Australia)
def to_sequences(df_AUS, seq_size=1):
    x = []
    y = []

    for i in range(len(df_AUS)-seq_size-1):
        #print(i)
        window = df_AUS[i:(i+seq_size), 0]
        x.append(window)
        y.append(df_AUS[i+seq_size, 0])
        
    return np.array(x),np.array(y)
    
seq_size = 10  # Number of time steps to look back 
#Larger sequences (look further back) may improve forecasting.

trainX, trainY = to_sequences(train, seq_size)
testX, testY = to_sequences(test, seq_size)



print("Shape of training set: {}".format(trainX.shape))
print("Shape of test set: {}".format(testX.shape))
#print("Shape of training set: {}".format(trainY.shape))
#print("Shape of test set: {}".format(testY.shape))

In [ ]:
# Reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print('Single LSTM with hidden Dense...')
model = Sequential()
model.add(LSTM(64, input_shape=(None, seq_size)))
model.add(Dense(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
#monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=20, 
#verbose=1, mode='auto', restore_best_weights=True)
model.summary()

In [ ]:
#Stacked LSTM with 1 hidden dense layer
#reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
#
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(None, seq_size)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(32))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

In [ ]:
#Bidirectional LSTM
#reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

##For some sequence forecasting problems we may need LSTM to learn
## sequence in both forward and backward directions
from keras.layers import Bidirectional
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(None, seq_size)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()


In [ ]:
trainX = trainX.reshape((trainX.shape[0], 1, 1, 1, seq_size))
testX = testX.reshape((testX.shape[0], 1, 1, 1, seq_size))

model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,1), activation='relu', input_shape=(1, 1, 1, seq_size)))
model.add(Flatten())
model.add(Dense(32))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

In [ ]:
start = timeit.default_timer()
model.fit(trainX, trainY, validation_data=(testX, testY),
          verbose=2, epochs=100)

stop = timeit.default_timer()
lama_eksekusi = stop - start
print("Lama eksekusi: ",lama_eksekusi,"detik")

In [ ]:
#Membuat Prediksi Data Training & Test
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
#Invert predictions back to prescaled values
#This is to compare with original input values
#Since we used minmaxscaler we can now use scaler.inverse_transform
#to invert the transformation.
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
#Menghitung Root Mean Squared Error (RMSE)
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))

testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
#Menghitung Mean Absolute Error (MAE)
mae = mean_absolute_error(trainY[0], trainPredict[:,0])
mae = mean_absolute_error(testY[0], testPredict[:,0])
print("Mean Absolute Error (MAE)_Train: ", mae)
print("Mean Absolute Error (MAE)_Test : ", mae)

In [ ]:
# shift train predictions for plotting
#we must shift the predictions so that they align on the x-axis with the original dataset. 
trainPredictPlot = np.empty_like(df_US)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[seq_size:len(trainPredict)+seq_size, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = np.empty_like(df_US)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(seq_size*2)+1:len(df_US)-1, :] = testPredict

In [ ]:
# plot baseline and predictions
plt.figure(figsize=(12,5), dpi=100)
plt.plot(scaler.inverse_transform(df_US))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()